In [2]:
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, RepeatVector, Dense

2024-02-29 05:40:51.053773: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 05:40:51.085834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 05:40:51.085882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 05:40:51.086916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 05:40:51.092421: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 05:40:51.093542: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
class CharacterTable(object):
	def __init__(self, chars):
		self.chars = sorted(set(chars))
		self.char_indices = dict((c,i) for i, c in enumerate(self.chars))
		self.indices_char = dict((i,c) for i, c in enumerate(self.chars))

	def encode(self, string, num_rows):
		x = np.zeros((num_rows, len(self.chars)))
		for i, char in enumerate(string):
			if char == ' ': 
				x[i, -1] = 1
				continue
			x[i, self.char_indices[char]] = 1
		return x

	def decode(self, x, calc_argmax=True):
		if calc_argmax:
			x = x.argmax(axis=-1)
		return ''.join([self.indices_char[i] for i in x])

In [4]:
training_size = 50000
digits = 5
hidden_size = 128
batch_size = 128

In [5]:
# This is a pattern to match against input: X+Y
question_maxlen = digits + 1 + digits
chars = '0123456789+- '

ctable = CharacterTable(chars)

questions = []
answers = []
expected = []
seen = set()

while len(questions) < training_size:
	f = lambda: int(''.join(np.random.choice(list('0123456789'))
		for i in range(np.random.randint(1, digits+1))))
	a, b = f(), f()

	key = tuple(sorted([a, b]))
	if key in seen:
		continue

	seen.add(key)

	question = f'{a}-{b}'
	padding = ' ' * (question_maxlen - len(question))

	question = question + padding

	answer = str(a - b)
	padding = ' ' * (question_maxlen - len(answer))

	questions.append(question)
	answers.append(answer)

In [6]:
q, a = questions[0], answers[0]

q, a

('13-774     ', '-761')

In [7]:
ctable.encode(q, question_maxlen)

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [8]:

# Encoding
x = np.zeros((len(questions), question_maxlen, len(chars)), dtype=bool)
y = np.zeros((len(questions), digits + 1, len(chars)), dtype=bool)
for i, question in enumerate(questions):
	x[i] = ctable.encode(question, question_maxlen)
for i, answer in enumerate(answers):
	y[i] = ctable.encode(answer, digits + 1)

In [9]:
print(x.shape, y.shape)

(50000, 11, 13) (50000, 6, 13)


In [10]:
print(x.astype(int)[0].T)
print(questions[0])

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 1]]
13-774     


In [11]:
print(y.astype(int)[0].T)
print(answers[0])

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
-761


In [12]:
# Make a random shuffling of the data
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Train test split (10% testing)
split = len(x) - len(x)//10

x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]

In [13]:
model = Sequential()
model.add(LSTM(100, input_shape=(question_maxlen, len(chars))))
model.add(RepeatVector(digits+1))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               45600     
                                                                 
 repeat_vector (RepeatVecto  (None, 6, 100)            0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 6, 50)             30200     
                                                                 
 time_distributed (TimeDist  (None, 6, 13)             663       
 ributed)                                                        
                                                                 
Total params: 76463 (298.68 KB)
Trainable params: 76463 (298.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
for iteration in range(1, 10):
	print(f"Iteration {iteration}")
	print('#'*25)
	model.fit(x_train, y_train,
			batch_size=batch_size,
			epochs=1,
			validation_data=(x_test, y_test))

Iteration 1
#########################


352/352 [==============================] - 8s 15ms/step - loss: 1.6668 - accuracy: 0.1376 - val_loss: 1.6338 - val_accuracy: 0.1549
Iteration 2
#########################
352/352 [==============================] - 5s 13ms/step - loss: 1.5501 - accuracy: 0.1685 - val_loss: 1.5121 - val_accuracy: 0.1907
Iteration 3
#########################
352/352 [==============================] - 5s 13ms/step - loss: 1.4482 - accuracy: 0.2101 - val_loss: 1.4134 - val_accuracy: 0.2248
Iteration 4
#########################
352/352 [==============================] - 5s 13ms/step - loss: 1.3664 - accuracy: 0.2380 - val_loss: 1.3467 - val_accuracy: 0.2504
Iteration 5
#########################
352/352 [==============================] - 5s 13ms/step - loss: 1.3046 - accuracy: 0.2609 - val_loss: 1.2910 - val_accuracy: 0.2726
Iteration 6
#########################
352/352 [==============================] - 5s 13ms/step - loss: 1.2553 - accuracy: 0.2827 - val_loss: 1.2508 - val_accuracy: 0.2882
Iteration 7
######

In [15]:
question_demo = q
answer_demo = a
x_demo = np.array([ctable.encode(question_demo, question_maxlen)])
y_demo = np.array([ctable.encode(answer_demo, digits + 1)])
print(x_demo)
prediction = model.predict(x_demo)
prediction = np.argmax(prediction, axis=-1)
# print(y_demo, prediction)
# print(y_demo.shape, prediction.shape)

correct = ctable.decode(y_demo[0])
print(correct)

guess = ctable.decode(prediction[0], calc_argmax=False)
print(guess)
# print(ctable.decode(y_demo), ctable.decode(prediction))

# rowx, rowy = x_test[np.array([0])], y_test[np.array([0])]
# print(rowx.shape, rowy.shape)
# print(x_demo.shape, y_demo.shape)
# # preds = model.predict_classes(rowx, verbose=0)
# preds = model.predict(rowx, verbose=0)
# preds = np.argmax(preds, axis=-1)

# question = ctable.decode(rowx[0])
# correct = ctable.decode(rowy[0])
# guess = ctable.decode(preds[0], calc_argmax=False)

# question, correct, guess

[[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]]


1/1 [==============================] - 1s 515ms/step
-761  
-76029
